In [49]:
#import time
import time
import datetime
import logging
import requests

import pandas as pd

import config

logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%H:%M:%S')
logger = logging.getLogger()
logger.setLevel(logging.INFO)


%run database.ipynb # import Database class

db = Database()

In [50]:
class DataFetcher:
    """ Fetch market data and historical data from the Polygon API.
    
    The frequency of requests is capped to avoid going beyond what Polygon 
    permits. The request is stalled upon reaching the maximum allowed frequency.
    
    """
    
    MAX_REQUEST_PER_MINUTE = 200
    STALL_TIME_UPON_MAX_REQUESTS = 3
    MAX_ATTEMPTS = 5
    
    def __init__(self):
        self._recent_requests = []
        
    def _request(self, url, params={}, attempts_left=DataFetcher.MAX_ATTEMPTS):
        params['apiKey'] = config.api_key
        result = requests.get(f'https://api.polygon.io{url}', params=params)

        if result.status_code == 200:
            json = result.json()
            if json.get('success', True):
                return json
        
        if attempts_left == 0:
            return None
        
        logging.error(
            f'Could not complete request {url} '
            f'(Error: {result.status_code}, attempts left: {attempts_left})'
        )
        time.sleep(5)
        return self._request(url, params, attempts_left-1)
    
    def get_ticker_details(self, ticker):
        # https://polygon.io/docs/get_v1_meta_symbols__stocksTicker__company_anchor
        url = f'/v1/meta/symbols/{ticker}/company'
        return self._request(url)
    
    
    def get_daily_trades(self, ticker, date, start_time=0):
        # https://polygon.io/docs/get_v2_ticks_stocks_trades__ticker___date__anchor
        
        TRADES_PER_REQUEST = 50000
        
        if type(date) == datetime.date:
            date = date.strftime('%Y-%m-%d')
            
        url = f'/v2/ticks/stocks/trades/{ticker}/{date}'
        params = {
            'timestamp': start_time,
            'limit': TRADES_PER_REQUEST
        }

        response = self._request(url, params)
        if response is None:
            return None

        # Exclude first trade in responses as it was already present in the 
        # previous request.
        trades = response['results'][int(start_time > 0):]

        # Repeat requests until all daily trades have been fetched.
        if response['results_count'] >= TRADES_PER_REQUEST:
            trades.extend(self.get_daily_trades(ticker, date, start_time=trades[-1]['t']))
        
        return trades
        
        
        
d = DataFetcher()
a = d.get_daily_trades('AAPL', datetime.date(2020, 1, 1))


21:01:59 Could not complete request /v2/ticks/stocks/trades/AAPL/2020-01-01 (Error: 404, attempts left: 5)
21:02:04 Could not complete request /v2/ticks/stocks/trades/AAPL/2020-01-01 (Error: 404, attempts left: 4)
21:02:10 Could not complete request /v2/ticks/stocks/trades/AAPL/2020-01-01 (Error: 404, attempts left: 3)
21:02:15 Could not complete request /v2/ticks/stocks/trades/AAPL/2020-01-01 (Error: 404, attempts left: 2)
21:02:20 Could not complete request /v2/ticks/stocks/trades/AAPL/2020-01-01 (Error: 404, attempts left: 1)


TypeError: 'NoneType' object is not subscriptable

In [47]:
print(a[50000-1])
print(a[50000])

{'t': 1577977011641553544, 'y': 1577977011641275029, 'q': 338517, 'i': '475', 'x': 15, 's': 3, 'c': [37], 'p': 297.75, 'z': 3}
{'t': 1577977011641704727, 'y': 1577977011639000000, 'f': 1577977011641668502, 'q': 338518, 'i': '11739', 'x': 4, 'r': 12, 's': 3, 'c': [37], 'p': 297.7342, 'z': 3}
